<a href="https://colab.research.google.com/github/mumorais/ENM_UnB-Analise_Modal/blob/main/_01_Fundamentos_1GdL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dado um sistema 1GdL amortecido forçado descrito por:
$$m \ddot{u}(t) + c \dot{u}(t) + k u(t) = F(t)$$
onde a massa $m = 10 kg$, o amortecimento $c = 4Nm/s$ e rigidez $k = 1kN/m$, e forçamento $F(t) =10 \delta(t)$. Lembre-se que $ \delta(t) $ é o delta de Dirac.

Determine:

OBS: A entrega pode ser redigida no moodle em conjunto com um arquivo anexo. 
Aconselha-se a entrega em arquivo notebook Colab (.ipynb).

1. A frequências natural $\omega_n$ e a razão de amortecimento $\xi$ do sistema 1GdL;

2.   Obtenha a resposta temporal analítica $u(t)$;



3.   Compare com a solução numérica por integração numérica;

4.   Obtenha a transformada de Fourier analítica $ \hat{u} (\omega) = \mathfrak{F}_{\omega} [u (t)] $; 

5.   Compare com a solução numérica por FFT.